In [107]:
import nltk
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
from IPython.core.display import HTML
import matplotlib.pyplot as plt
stopwords = nltk.corpus.stopwords.words('english')

In [123]:
def freq(word, tokens):
    return tokens.count(word)

def word_count(tokens):
    return len(tokens)

def tf(word, tokens):
    return (freq(word, tokens) / float(word_count(tokens)))

def getFreqByDoc(doc):
    #get tokens
    tokens = nltk.word_tokenize(str(doc[1]) + ' ' + str(doc[2]))
    
    #get bitokens
    bi_tokens =  nltk.bigrams(tokens)
    bi_tokens = [' '.join(token).lower() for token in bi_tokens]
    bi_tokens = [token for token in bi_tokens if token not in stopwords]

    tokens = [token.lower() for token in tokens if len(token) > 2]
    tokens = [token for token in tokens if token not in stopwords]

    #merge tokens and bi
    alltokens = []
    alltokens.extend(tokens)
    alltokens.extend(bi_tokens)

    wc = word_count(tokens)
    olist = []
    for token in (alltokens):
        f = freq(token, alltokens)
        ilist = [doc[0], token, f, wc, f/float(wc), doc[3], doc[5]]
        olist.append(ilist)
    df = pd.DataFrame(olist, columns=['doc','word', 'frequency', 'word_count', 'tf','source', 'sample_type'])
    with open('C:\\datascience\\datatest.csv', 'a') as f:
        df.to_csv(f, header=False)    

    return df

def createFreqDf(sdf):
    #pass a sample df
    #make a list so that it can be iterated
    mfgDfSampleList = sdf.as_matrix().tolist()
    mfgFreqDfList = []
    for mfg in mfgDfSampleList:
        df = getFreqByDoc(mfg)
        mfgFreqDfList.append(df)

    mfgFreqDf = pd.concat(mfgFreqDfList)
    return mfgFreqDf

In [119]:
#reading the abstracts in to a DF
mfgDf = pd.DataFrame.from_csv('../../data/positive_data.csv', index_col=None)
nsfDf = pd.DataFrame.from_csv('../../data/negative_data.csv', index_col=None)

In [120]:
#draw samples
mfgDfSample = mfgDf.sample(10)
mfgDfSample["sample_type"] = "MFG"

'''nsfDfSample = nsfDf.sample(5)
nsfDfSample["sample_type"] = "NON-MFG"'''

'nsfDfSample = nsfDf.sample(5)\nnsfDfSample["sample_type"] = "NON-MFG"'

In [121]:
mfgDfSample.head()

,id,title,abstract,event_name,event_year,sample_type
2185,NSF_20140101_1462280,CHS: Medium: Collaborative Research: Immediate...,American Sign Language (ASL) is a primary mean...,NSF,2014.0,MFG
4080,CIRP_20120101_383-386,Raw part characterisation and automated alignm...,Large raw parts require a long time consuming ...,CIRP,2012.0,MFG
2627,NSF_20150101_1536918,Household-Level Use of Autonomous Vehicles: Mo...,"With recent development in vehicle automation,...",NSF,2015.0,MFG
4358,CIRP_20140101_058-1205,Alternation of analysis and synthesis for conc...,Concept generation involves both analysis and ...,CIRP,2014.0,MFG
466,NSF_20130101_1316583,"Workshop on Humanitarian Logistics Research, A...",This grant provides funding to host a workshop...,NSF,2013.0,MFG


In [124]:
mfgf = createFreqDf(mfgDfSample)
#print(mfgf)
#nsff = createFreqDf(nsfDfSample)
#print(nsfgf)    
mfgf.head()

,doc,word,frequency,word_count,tf,source,sample_type
0,NSF_20140101_1462280,chs,1,302,0.003311,NSF,MFG
1,NSF_20140101_1462280,medium,1,302,0.003311,NSF,MFG
2,NSF_20140101_1462280,collaborative,1,302,0.003311,NSF,MFG
3,NSF_20140101_1462280,research,2,302,0.006623,NSF,MFG
4,NSF_20140101_1462280,immediate,3,302,0.009934,NSF,MFG


In [113]:
mfgf.head()
#nsff.count()
#mfgDfSample.count()
#nsfDfSample.count()

,doc,word,frequency,word_count,tf,source,sample_type
0,CIRP_20090101_2009-58-2-0647-E,ion,4,53,0.075472,CIRP,MFG
1,CIRP_20090101_2009-58-2-0647-E,beam,4,53,0.075472,CIRP,MFG
2,CIRP_20090101_2009-58-2-0647-E,focused,2,53,0.037736,CIRP,MFG
3,CIRP_20090101_2009-58-2-0647-E,ion,4,53,0.075472,CIRP,MFG
4,CIRP_20090101_2009-58-2-0647-E,beam,4,53,0.075472,CIRP,MFG


In [114]:
mfgf_grp = mfgf.groupby(["word", "doc"]).size()

In [115]:
mfgf_grp.sample(100)

word                 doc                            
the traditional      CIRP_20080101_2008-57-1-0227-E     1
analysis             NSF_20130101_1344017               1
inability            NSF_20140101_1407033               1
meeting .            NSF_20130101_1342469               1
occasions            NSF_20140101_1407033               1
methods for          NSF_20140101_1453651               1
science              NSF_20140101_1453651               1
projects             NSF_20140101_1453651               1
key topic            NSF_20140101_1453651               1
blast loads          NSF_20130101_1342469               1
and ploughing        CIRP_20090101_2009-58-1-0049-C     1
balance between      NSF_20140101_1407033               1
given other          NSF_20140101_1453651               1
performed quickly    NSF_20140101_1407033               1
based                CIRP_20150101_064-1305             1
rebuilding           NSF_20140101_1407033               1
career :           

In [116]:
d = mfgf
d = d[(d.doc == "NSF_20150101_1537648") & (d.word == "graphene")]
d

,doc,word,frequency,word_count,tf,source,sample_type
